In [1]:
# pbp_parse.py
#
# parse JSON, make game structure, convert to CSV.
# JSON 데이터를 읽어와 게임 상황을 구현하고
# gameday line 하나에 맞는 text row를 생성, CSV 파일에 저장한다.

import os
import json
from collections import OrderedDict
import regex
import csv

# custom library
from utils import print_progress

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import sys
game_id = '20190331SKWO02019'
pl = '_pitching.csv'
bl = '_batting.csv'
rl = '_relay.csv'

encoding = 'cp949' if sys.platform == 'win32' else 'utf8'
pdf = pd.read_csv(f'pbp_data/2019/3/{game_id}{pl}', encoding=encoding)
bdf = pd.read_csv(f'pbp_data/2019/3/{game_id}{bl}', encoding=encoding)
rdf = pd.read_csv(f'pbp_data/2019/3/{game_id}{rl}', encoding=encoding)
rdf = rdf.sort_values(['textOrder', 'seqno'])

In [3]:
tcol = ['textOrder', 'seqno', 'text', 'type', 'stuff', 'pitchNum',
        'pitchResult', 'pitchId', 'speed', 'referee', 'stadium', 'inn',
        'ballcount', 'stance']
pcol = ['pitchId', 'speed', 'crossPlateX',
        'vx0', 'vy0', 'vz0', 'x0', 'z0', 'ax', 'ay', 'az',
        'topSz', 'bottomSz']
scol = ['outPlayer', 'inPlayer', 'shiftPlayer']

In [4]:
away_order = []
home_order = []
away_field = {}
home_field = {}

abat = bdf.loc[bdf.homeaway == 'a']
hbat = bdf.loc[bdf.homeaway == 'h']

apit = pdf.loc[pdf.homeaway == 'a']
hpit = pdf.loc[pdf.homeaway == 'h']

abat_seqno_min = abat.groupby('batOrder').seqno.min().tolist()
hbat_seqno_min = hbat.groupby('batOrder').seqno.min().tolist()

for i in range(9):
    aCode = abat.loc[(abat.batOrder == i+1) & (abat.seqno == abat_seqno_min[i])].pCode.values[0]
    aName = abat.loc[(abat.batOrder == i+1) & (abat.seqno == abat_seqno_min[i])].name.values[0]
    hCode = hbat.loc[(hbat.batOrder == i+1) & (hbat.seqno == hbat_seqno_min[i])].pCode.values[0]
    hName = hbat.loc[(hbat.batOrder == i+1) & (hbat.seqno == hbat_seqno_min[i])].name.values[0]
    aPlayer = {'name': aName, 'code': aCode}
    hPlayer = {'name': hName, 'code': hCode}
    away_order.append(aPlayer)
    home_order.append(hPlayer)
    
    aPos = abat.loc[abat.pCode == aCode].posName.values[0]
    hPos = hbat.loc[hbat.pCode == hCode].posName.values[0]
    away_field[aPos] = aPlayer
    home_field[hPos] = hPlayer

aPitcher = {'name': apit.iloc[0]['name'], 'code': apit.iloc[0].pCode}
hPitcher = {'name': hpit.iloc[0]['name'], 'code': hpit.iloc[0].pCode}
away_field['투수'] = aPitcher
home_field['투수'] = hPitcher

In [5]:
game_date = game_id[:8]
away = game_id[8:10]
home = game_id[10:12]
stadium = rdf.stadium.drop_duplicates().values[0]
referee = rdf.referee.drop_duplicates().values[0]

In [6]:
rdf.columns

Index(['textOrder', 'seqno', 'text', 'type', 'stuff', 'pitchNum',
       'pitchResult', 'pitchId', 'speed', 'referee', 'stadium', 'outPlayer',
       'inPlayer', 'shiftPlayer', 'inn', 'ballcount', 'crossPlateX', 'topSz',
       'vy0', 'vz0', 'vx0', 'z0', 'ax', 'x0', 'ay', 'az', 'bottomSz',
       'stance'],
      dtype='object')

# Structure

Top-Bottom 식으로 밑그림을 그리고 가보자.

## Read Files

경기 별로 파일은 3가지가 저장된다.
1. pbp_data/{Year}/{Month}/{gameID}_relay.csv
2. pbp_data/{Year}/{Month}/{gameID}_batting.csv
3. pbp_data/{Year}/{Month}/{gameID}_pitching.csv



# 1. `relay` 중계 파일 구조

`csv` 형식으로 저장했는데 각 컬럼의 의미는 이렇다.

columns| definition
-|-
textOrder| 텍스트 순번(타석, 메시지 단위로 구분)
seqno| 같은 `textOrder` 안에서 출력 순서
text | 중계 텍스트 내용
type | 텍스트 타입
stuff | 구종
pitchNum | 타석 투구수(신뢰 x)
pitchResult | 스트(T)/볼(B)/파울(F)/헛스윙(S)
pitchId | 트래킹데이터 고유값 ID
speed | 문자중계 찍히는 구속
referee | 구심
stadium | 구장
outPlayer | 교체시/나가는 선수
inPlayer | 교체시/들어온 선수
shiftPlayer | 포변/포변 선수
inn | 이닝(신뢰 x)
ballcount | 타석 투구수(신뢰 x)
기타 | vx0, vy0, ... 트래킹 데이터

## type 의미

- `0` : x회초(말) xx 공격
- `1` : 투구 (x구 ...)
- `2` : 교체
- `7` : 시스템 메시지 (비디오 판독, ...)
- `8` : x번타자 xxx
- `13` : 타자주자 타석 결과(득점 없는 타석)
- `14` : 주자 주루/아웃(득점 없는 타석)
- `23` : 타자주자 타석 결과(득점 있는 타석)
- `24` : 주자 주루/아웃(득점 있는 타석)
- `44` : 파울 에러 (pass)
- `99` : 경기 종료 메시지

# 2. `batting` 파일 구조 / 야수 관련 `object` 설정

`csv` 형식이며 각 컬럼 의미는 이렇다.

columns | definition
-|-
name | 선수 이름
pCode | 선수 코드
pos | 포지션(한글) - 선발선수는 선발시 포지션, 나머지는 경기종료시 포지션
hitType | 투타
seqno | 해당 타순 선수들의 출전 순서(낮으면 선발)
batOrder | 타순
homeaway | `h`는 홈, `a`는 어웨이
ab, hit, ... | 경기 기록

## 초기 `object` 설정

`csv` 파일을 `pd.DataFrame` 형식으로 로드해둔다.

로드 이후 다음 순서로 경기 시작시 배팅 오더/수비 라인업 오브젝트를 구성한다.

1. `batOrder` 순으로 순회한다.
2. 각 `batOrder`에서 `seqno` 값이 가장 낮은 row를 뽑아 라인업 오더에 넣는다.
  2-1. `name`, `pCode`만 넣어둔다. 세부 정보 필요시 `pCode`로 `DataFrame`을 조회한다.
  2-2. 해당 row의 포지션으로 선발 수비 라인업도 구성한다.
3. 수비라인업의 투수 설정을 한다. (#3. `pitching` 파일 구조 참조)

## 조회하는 경우
경기 중에 배팅 오더/수비 라인업 오브젝트를 조회하는 경우는 다음이 있다.

1. 선수 교체시
  1-1. 대타 : 배팅 오더 교체, 나간선수 수비 라인업 `nullify`
  1-2. 투교 : 수비 라인업 `refresh`
  1-3. 포변 : 수비 라인업 `refresh`
2. 1구 단위 row 기록할 때 : 수비 라인업 조회하고 기록
3. 타석 바뀔 때: 타순 보고 선수 대조

# 3. `pitching ` 파일 구조 / 투수 관련 `object` 설정

`csv` 형식이며 각 컬럼 의미는 이렇다.

columns | definition
-|-
name | 선수 이름
pCode | 선수 코드
hitType | 투타
seqno | 출전 순서(낮으면 선발)
homeaway | `h`는 홈, `a`는 어웨이
inn, run, er, ... | 경기 기록


## 초기 `object` 설정

`csv` 파일을 `pd.DataFrmae` 형식으로 로드해둔다.

로드 이후 다음 순서로 경기 시작시 배팅 오더/수비 라인업 오브젝트를 구성한다.

1. 야수쪽 수비 라인업 설정을 마친뒤 투수도 수비 라인업에 설정한다.
  1-1. 야수와 마찬가지로 `name`, `pCode`만 넣어둔다.
  1-2. `seqno` 가장 낮은 투수를 집어넣는다.

## 조회하는 경우

경기 중에 투수 라인업을 조회하는 경우는 다음이 있다.

1. 선수 교체시
  1-1. 대타: 아주 가끔 투수가 대타로 나갈 때 있다.
  1-2. 투교: 수비 라인업 `refresh`
2. 1구 단위 row 기록할 때 : 수비 라인업 조회하고 기록

# 로드 이후 순서

1. `textOrder`로 중계 텍스트를 나눈다.
2. `seqno` 순서로 텍스트를 로드, 파싱한다.
3. 파싱 결과에 따라 game status를 변경
4. 텍스트 전체 파싱, 진행 끝나면 csv로 저장

# 파싱 순서

1. 타석 단위로 스택 처리, `type` 따라 달라짐
    - `type`에 따라서...
        - `0` : 이닝 status refresh
        - `1` : 투구 결과 및 데이터 기록
        - `2` : 교체 반영 -> 배팅 오더, 수비 라인업
        - `7` : 시스템 메시지
            - continue
        - `8` : x번타자 xxx -> 타석 status refresh
        - `13` : 타자주자 타석 결과(득점 없는 타석)
            - 타석 stack 생성
            - 타석 결과 나왔다는 flag True로 바꾸기
            - base 결과, out 결과, score 결과 stack에 쌓는다
            - **곧바로 출력하지 않는다**: PA 끝까지 stack에 쌓아서 주루 결과까치 처리 후 출력
            - 결과도 즉각 game status에 반영하지 않는다            
        - `14` : 주자 주루/아웃(득점 없는 타석)
            - 타석 결과 flag 확인 : False인 경우 새로 stack 생성
                - 타석 도중에 나올 수 있기 때문(도루 성공&실패, 견제사, 보크, 폭투 등)
            - stack 다 쌓은 다음에 처리
        - `23` : 타자주자 타석 결과(득점 있는 타석)
            - `13`과 동일, '홈인' 메시지는 득점 반영 / 주자 베이스 처리
        - `24` : 주자 주루/아웃(득점 있는 타석)
            - `14`와 동일, '홈인' 메시지는 득점 반영 / 주자 베이스 처리
        - `44` : 파울 에러 (pass)
            - continue
        - `99` : 경기 종료 메시지
            - continue
2. 스택 처리
    - 

# 타석 결과, 주자 주루 stack 처리

# 타석 결과 종류 정리

keyword | 간단 분류 | 상세 분류
-|-|-
삼진 | 삼진 | 삼진
볼넷 | 볼넷 | 삼진
자동 고의 4구 | 고의 4구 | 고의 4구
고의4구 | 고의 4구 | 고의 4구
몸에 | 몸에 맞는 공 | 몸에 맞는 공
1루타 | 안타 | 안타
내야안타 | 내야 안타 | 내야 안타
번트안타 | 번트 안타 | 번트 안타
안타 | 안타 | 안타
2루타 | 2루타 | 2루타
3루타 | 3루타 | 3루타
홈런 | 홈런 | 홈런
낫아웃 폭투 | 낫아웃 출루 | 낫아웃 폭투
낫아웃 포일 | 낫아웃 출루 | 낫아웃 포일
낫 아웃 | 삼진 | 낫아웃 삼진
낫아웃 다른주자 수비 실책 | 낫아웃 출루 | 낫아웃 다른 주자 수비 실책
낫아웃 다른주자 수비 | 낫아웃 출루 | 낫아웃 다른 주자 포스 아웃
땅볼로 출루 | 포스 아웃 | 포스 아웃
플라이 아웃 | 필드 아웃 | 필드 아웃
인필드 | 필드 아웃 | 인필드 플라이
파울플라이 | 필드 아웃 | 파울 플라이 아웃
라인드라이브 아웃 | 필드 아웃 | 라인드라이브 아웃
번트 아웃 | 필드 아웃 | 번트 아웃
병살타 | 병살타 | 병살타
희생번트 아웃 | 희생 번트 | 희생 번트
희생플라이 아웃 | 희생 플라이 | 희생 플라이
희생플라이아웃 | 희생 플라이 | 희생 플라이
쓰리번트 | 삼진 | 쓰리번트 삼진
타구맞음 | 필드 아웃 | 타구맞음 아웃
희생번트 실책 | 실책 | 실책
희생번트 야수선택 | 야수 선택 | 야수 선택
야수선택 | 야수 선택 | 야수 선택
실책 | 실책 | 실책
타격방해 | 타격 방해 | 타격 방해
삼중살 | 삼중살 | 삼중살
부정타격 | 필드 아웃 | 부정 타격 아웃
번트 | 번트 안타 | 안타

In [27]:
rdf.loc[rdf.textOrder == 12][tcol]

,textOrder,seqno,text,type,stuff,pitchNum,pitchResult,pitchId,speed,referee,stadium,inn,ballcount,stance
68,12,68,5번타자 나주환,8,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN
69,12,69,1구 볼,1,투심,1.0,B,190331_142746,142.0,우효동,고척,2.0,1.0,R
70,12,70,2구 볼,1,투심,2.0,B,190331_142807,142.0,우효동,고척,2.0,2.0,R
71,12,71,3구 볼,1,투심,3.0,B,190331_142835,141.0,우효동,고척,2.0,3.0,R
72,12,72,4구 스트라이크,1,슬라이더,4.0,T,190331_142903,135.0,우효동,고척,2.0,4.0,R
73,12,73,5구 타격,1,투심,5.0,H,190331_142926,141.0,우효동,고척,2.0,5.0,R
74,12,74,나주환 : 3루수 땅볼 아웃 (3루수->1루수 송구아웃),13,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN
75,12,75,1루주자 이재원 : 2루까지 진루,14,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN


In [97]:
class game_status:
    def __init__ (self):
        self.score_home = 0
        self.score_away = 0
        self.balls = 0
        self.strikes = 0
        self.outs = 0
        self.on_1b = None
        self.on_2b = None
        self.on_3b = None
        self.inn = 0
        self.inn_topbot = 't'
        self.game_date = None
        self.away = None
        self.home = None
        self.stadium = None
        self.referee = None
        self.pa_number = 0
        self.pitch_number = 0
        self.away_field = None
        self.away_order = None
        self.home_field = None
        self.home_order = None
        self.pitchData = None # pointer

    def setting(self, game_id, text_df, away_lineup_df, home_lineup_df):
        self.game_date = game_id[:8]
        self.away = game_id[8:10]
        self.home = game_id[10:12]
        self.stadium = text_df.stadium.drop_duplicates().values[0]
        self.referee = text_df.referee.drop_duplicates().values[0]
        
        away_order = []
        home_order = []
        away_field = {}
        home_field = {}

        abat = bdf.loc[bdf.homeaway == 'a']
        hbat = bdf.loc[bdf.homeaway == 'h']

        apit = pdf.loc[pdf.homeaway == 'a']
        hpit = pdf.loc[pdf.homeaway == 'h']

        abat_seqno_min = abat.groupby('batOrder').seqno.min().tolist()
        hbat_seqno_min = hbat.groupby('batOrder').seqno.min().tolist()

        for i in range(9):
            aCode = abat.loc[(abat.batOrder == i+1) & (abat.seqno == abat_seqno_min[i])].pCode.values[0]
            aName = abat.loc[(abat.batOrder == i+1) & (abat.seqno == abat_seqno_min[i])].name.values[0]
            hCode = hbat.loc[(hbat.batOrder == i+1) & (hbat.seqno == hbat_seqno_min[i])].pCode.values[0]
            hName = hbat.loc[(hbat.batOrder == i+1) & (hbat.seqno == hbat_seqno_min[i])].name.values[0]
            aPlayer = {'name': aName, 'code': aCode}
            hPlayer = {'name': hName, 'code': hCode}
            away_order.append(aPlayer)
            home_order.append(hPlayer)

            aPos = abat.loc[abat.pCode == aCode].posName.values[0]
            hPos = hbat.loc[hbat.pCode == hCode].posName.values[0]
            away_field[aPos] = aPlayer
            home_field[hPos] = hPlayer

        aPitcher = {'name': apit.iloc[0]['name'], 'code': apit.iloc[0].pCode}
        hPitcher = {'name': hpit.iloc[0]['name'], 'code': hpit.iloc[0].pCode}
        away_field['투수'] = aPitcher
        home_field['투수'] = hPitcher
        
        self.away_order = away_order
        self.away_field = away_field
        self.home_order = home_order
        self.home_field = home_field
    
    def 

In [73]:
lenTexts = len(rdf.textOrder.drop_duplicates())
for i in range(lenTexts):
    firstRow = rdf.loc[rdf.textOrder == i].head(1)
    if firstRow.text.values[0].find('대타') > -1:
        print(i)
        display(firstRow[['text', 'type']].values[0].tolist())

66


['대타 정의윤', 8]

77


['대타 이지영', 8]

85


['대타 김규민', 8]

In [133]:
def parseBatterResult(text):
    return text
def parseRunnerResult(text):
    return text

def parseTextWithType(self, textdf):
    textType = textdf.type
    text = textdf.text
    if textType == 0:
        # 이닝
        inn = int(text.split('회')[0])
        topbot = text.split('회')[1].split(' ')[0]
        topbot = 't' if topbot == '초' else 'b'
        print(inn, topbot)
    elif textType == 1:
        # 투구
        pID = textdf.pitchId
        print(text, pID)
    elif textType == 2:
        # 교체
        # shift, outPlayer, inPlayer
        if np.isnan(textdf.shiftPlayer):
            outPlayer = textdf.outPlayer
            inPlayer = textdf.inPlayer
            # 포지션+이름으로 away / home field, order에서 선수 code 찾기(이름만으로 찾으면 동명이인)
            print(text, outPlayer, inPlayer)
        else:
            shiftPlayer = text.split(' ')[1]
            beforePos = text.split(' ')[0]
            afterPos = text.split(' ')[3].split('(')[0]
            # 포지션+이름으로 away / home field, order에서 선수 code 찾기(이름만으로 찾으면 동명이인)
            # 해당 선수 교체
            print(text, shiftPlayer, beforePos, afterPos)
        

In [186]:
#for i in range(rdf.shape[0]):
for i in range(10):
    j = np.random.randint(rdf.loc[rdf.type == 13].shape[0])
    textdf = rdf.loc[rdf.type == 13].iloc[j]
    textType = textdf.type
    text = textdf.text
    if textType == 0:
        # 이닝
        inn = int(text.split('회')[0])
        topbot = text.split('회')[1].split(' ')[0]
        topbot = 't' if topbot == '초' else 'b'
        print(inn, topbot)
    elif textType == 1:
        # 투구
        pID = textdf.pitchId
        print(text, pID)
    elif textType == 2:
        # 교체
        # 대타, 수비교체, 포지션변경
        # shift, outPlayer, inPlayer
        if np.isnan(textdf.shiftPlayer):
            outPCode = textdf.outPlayer
            inPCode = textdf.inPlayer
            outPos = text.split(' ')[0]
            outPlayer = text.split(' ')[1]
            inPos = text.split(' ')[3]
            inPlayer = text.split(' ')[4]
            # 포지션+이름으로 away / home field, order에서 선수 code 찾기(이름만으로 찾으면 동명이인)
            print(outPos, outPlayer, outPCode, inPos, inPlayer, inPCode)
        else:
            shiftPlayer = text.split(' ')[1]
            beforePos = text.split(' ')[0]
            afterPos = text.split(' ')[3].split('(')[0]
            # 포지션+이름으로 away / home field, order에서 선수 code 찾기(이름만으로 찾으면 동명이인)
            # 해당 선수 교체
            print(shiftPlayer, beforePos, afterPos)
    elif textType == 7:
        # 시스템 메시지
        continue
    elif textType == 8:
        # x번타자 xxx
        player = text.split(' ')[1]
        if text.find('번타자') > 0:
            order = text.split('번')[0]
            print(order, player)
        else:
            print('대타', player)
    elif textType == 13:
        # 타자주자 타석 결과(득점x 타석)
        player = text.split(' ')[0]
        result = text.split(':')[1].strip()
        print(player, result)
        continue
    elif textType == 14:
        # 주자 주루/아웃(득점x 타석)
        continue
    elif textType == 23:
        # 타자주자 타석 결과(득점x 타석)
        continue
    elif textType == 24:
        # 주자 주루/아웃(득점x 타석)
        continue
    elif textType == 44:
        # 파울 에러 메시지
        continue
    elif textType == 99:
        # 경기 종료
        continue

김강민 우중간 2루타
김성현 유격수 플라이 아웃
김하성 3루수 앞 땅볼로 출루
이정후 중견수 오른쪽 2루타
허정협 삼진 아웃
허도환 볼넷
김성현 유격수 플라이 아웃
허정협 1루수 파울플라이 아웃
최정 몸에 맞는 볼
이정후 유격수 땅볼 아웃 (유격수->1루수 송구아웃)


In [135]:
sdf = rdf.loc[rdf.type == 2]
for i in range(sdf.shape[0]):
    parseTextWithType(None, sdf.iloc[i])

투수 다익손 : 투수 박민호 (으)로 교체 69861.0 64893.0
투수 박민호 : 투수 하재훈 (으)로 교체 64893.0 69813.0
1루주자 허정협 : 대주자 박정음 (으)로 교체 65399.0 62353.0
투수 최원태 : 투수 오주원 (으)로 교체 65320.0 74359.0
대주자 박정음 : 지명타자(으)로 수비위치 변경 박정음 대주자 지명타자
투수 오주원 : 투수 이보근 (으)로 교체 74359.0 75342.0
9번타자 김성현 : 대타 정의윤 (으)로 교체 76802.0 75151.0
1루주자 정의윤 : 대주자 강승호 (으)로 교체 75151.0 63123.0
투수 이보근 : 투수 김성민 (으)로 교체 75342.0 67828.0
투수 하재훈 : 투수 서진용 (으)로 교체 69813.0 61895.0
대주자 강승호 : 유격수(으)로 수비위치 변경 강승호 대주자 유격수
9번타자 주효상 : 대타 이지영 (으)로 교체 66354.0 79456.0
투수 서진용 : 투수 김택형 (으)로 교체 61895.0 65343.0
7번타자 박정음 : 대타 김규민 (으)로 교체 62353.0 62356.0
투수 김택형 : 투수 이승진 (으)로 교체 65343.0 64805.0
투수 김성민 : 투수 김상수 (으)로 교체 67828.0 76430.0
대타 김규민 : 지명타자(으)로 수비위치 변경 김규민 대타 지명타자
대타 이지영 : 포수(으)로 수비위치 변경 이지영 대타 포수
투수 이승진 : 투수 강지광 (으)로 교체 64805.0 79130.0
투수 김상수 : 투수 김동준 (으)로 교체 76430.0 62360.0
지명타자 이재원 : 포수(으)로 수비위치 변경 이재원 지명타자 포수
포수 허도환 : 투수 김태훈 (으)로 교체 77243.0 79847.0
1루주자 장영석 : 대주자 김혜성 (으)로 교체 79334.0 67304.0


`type`
- `0` : x회초(말) xx 공격
- `1` : 투구 (x구 ...)
- `2` : 교체
- `7` : 시스템 메시지 (비디오 판독, ...)
- `8` : x번타자 xxx
- `13` : 타자주자 타석 결과(득점 없는 타석)
- `14` : 주자 주루/아웃(득점 없는 타석)
- `23` : 타자주자 타석 결과(득점 있는 타석)
- `24` : 주자 주루/아웃(득점 있는 타석)
- `44` : 파울 에러 (pass)
- `99` : 경기 종료 메시지

`pitchResult`
- `T` : s __T__ rike 스트라이크
- `F` : __F__oul 파울
- `S` : __S__wing 헛스윙
- `B` : __B__all 볼
- `H` : __H__it 타격

- `pitchNum` : 문자중계 텍스트(`textOptionList`)에 포함되는 투구수, 정합성 확인 안됨, 신뢰하지 말 것
- `ballcount` : pitch data(`ptsOptionList`)에 포함되는 투구수, `ptsPitchId` 오류 있을 때는 1개씩 밀리는 문제, 신뢰하지 말 것
- `inn` : 문자중계와 pitch data에 모두 포함, 지금은 Pitch data에 있는걸 가져오는 중. 신뢰하지 말 것